# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Extract all rows of data
all_data = soup.find_all("table", class_ = "table")
all_data

In [ ]:
#Create a column list

headers = soup.find_all("th")
headers
column_list = []
for header in headers:
    column = header.text
    column_list.append(column)
    
column_list
    


### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
row_data = []
row_mars_data = soup.find_all("tr", class_ = "data-row")
#row_mars_data
# Loop through the scraped data to create a list of rows

for row in row_mars_data:
    td = row.find_all('td')
    row = [col.text for col in td]
    row_data.append(row)
row_data    

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
df = pd.DataFrame(row_data, columns = column_list)

In [ ]:
# Confirm DataFrame was created successfully
df.head()

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
df.dtypes

In [ ]:
# Change data types for data analysis
df_mars = df.astype({'terrestrial_date': 'datetime64','sol': 'int64','ls': 'int64','month':'int64',
           'min_temp':'float64','pressure':'float64'})

In [ ]:
# Confirm type changes were successful by examining data types again
df_mars.dtypes


### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
df_mars.groupby(['month'])['month'].count()

In [ ]:
# to get the no. of rows and columns
df_mars.shape

In [ ]:
# 2. How many Martian days' worth of data are there?
df_mars['id'].count()

In [ ]:
# 3. What is the average low temperature by month?
df_min = df_mars.groupby(['month'])['min_temp'].mean()
df_min


In [ ]:
# Plot the average temperature by month
plt.bar(df_min.index, df_min.values)
plt.xlabel('month')
plt.ylabel('Temperatures in C')
plt.xticks(ticks = range(1,13,1))
plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location

df_min.sort_values(ascending = True).plot(kind = 'bar')
plt.show()


In [ ]:
# 4. Average pressure by Martian month
df_pressure = df_mars.groupby(['month'])['pressure'].mean()
df_pressure

In [ ]:
# Plot the average pressure by month
df_pressure.sort_values(ascending = True).plot(kind = 'bar')
plt.xlabel('Month')
plt.ylabel('Atmospheric Pressure')
plt.show()


In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
x_axis = df_mars.index
y_axis = df_mars['min_temp']
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(x_axis,y_axis,
           )
ax.set(xlabel="Number of terrestrial dates", ylabel="Temperature in Celcius")
plt.show()
      

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
df_mars.to_csv('./Output/df_mars.csv')
df_min.to_csv('./Output/df_min.csv')
df_pressure.to_csv('./Output/df_pressure.csv')

In [ ]:
browser.quit()